# PREREQUISITES
You'll need nbconvert. Run the cell below

In [ ]:
#! pip install nbconvert

### Imports
If something doesn't work, pip install it

In [ ]:
import os
import smtplib
from email.message import EmailMessage
import re
import codecs

# CODE
Defining student class

In [ ]:
class Student:

    def __init__(self, vunet_id: str):
        self.vunet_id = vunet_id

        # Turns out, you don't need to scavenge the student vu email
        # you can just use the vunet id, and it's treated the same
        self.email_address = self.vunet_id + "@student.vu.nl"


Defining the rest of everything else

In [ ]:
# Let me preface this by saying that I have frankenstein'd this code together from the various iterations
# of this project. Some of the code isn't used enough to warrant a function, but I'm too lazy to remove it.
# It works, it's readable, and it's not too slow. That's all I care about.

def htmlConvert(
        ta_name: str = None,
        ta_email: str = None,
        ta_email_password: str = None,
        assignment_number: str = None,
        dir_path : str = os.path.dirname(os.getcwd()), 
        file_convert: bool = True, 
        send_email: bool = False,
        test_mode: bool = False,
        go_to_vunet_id: str = None,
        id_mode: str = "SCRAPE"
        ) -> None:
    """
    In a given directory, goes through all folders (1 layer), and converts all ipynb files to html files
    :param ta_name: TA name - used in email
    :param ta_email: TA email
    :param ta_email_password: email password
    :param assignment_number: assignment number - should be formatted as "Assignment 2" or "Midterm"
    :param dir_path: The path of the directory to convert the files in (optional). Default is the current directory.
    :param file_convert: Whether or not to convert the files. Default is True.
    :param send_email: Whether or not to send the emails. Default is True.
    :param test_mode: Whether or not to run in test mode. Default is False. -> disables sending emails and converting files
    :param go_to_vunet_id: The vunet id to start at. Default is None. -> starts at the first vunet id. Use if the cell crashes to avoid sending emails twice.
    :param id_mode: The mode to get the vunet id. Default is "SCRAPE". -> gets the vunet id from the file itself. Use "FODLER" to get the vunet id from the file path.
    """

    #if not file_convert and not send_email:
    list_of_sent_emails: list[str] = []
    list_of_unsent_emails: list[str] = []


    # Check if the directory path is valid
    if not os.path.isdir(dir_path):
        raise ValueError("Invalid directory path")

    # Idiot proofing
    assert id_mode in ["FOLDER", "SCRAPE"]


    if send_email:

        assert ta_email != None
        assert ta_email_password != None
        assert ta_name != None
        assert assignment_number != None
        

        go_to_latest = True
        if go_to_vunet_id is not None:
            go_to_latest = False

    print("Starting...\n")
    
    # Get a list of all the folders and their files in the directory
    dir_list : list[tuple[list]] = [x for x in os.walk(dir_path)]

    # Convert all the ipynb files to html files
    for dir in dir_list:
        dir_name : str = dir[0]
        if dir_name.endswith(".ipynb_checkpoints"): continue
        if dir_name.endswith("ipynb-to-html-converter"): 
            continue
            
        
        # runs through all files per folder, converts all ipynb files to html files
        for file_name in dir[2]:

            if not file_name.endswith(".ipynb"):
                continue
            
            file_path : str = '"'+os.path.join(dir_name, file_name)+'"'

            if id_mode == "SCRAPE":
                f = codecs.open(file_path.strip('"'), 'r')
                for line in f:
                    
                    if "VUnet" in line:
                        # Cursed vunet id scraper
                        extracted_id = re.search(r'\b[a-zA-Z]{3}\d{3}\b', line)
                        if extracted_id is not None:
                            vunet_id = extracted_id.group().lower()
                            break
                else:
                    vunet_id = None


            elif id_mode == "FOLDER":
                vunet_id = file_path.split("\\")[-2]

                if len(vunet_id) != 6:
                    vunet_id = None

            if file_convert:

                if vunet_id is None:
                    undefined = "undefined"
                    print(f"\nConverting file for {undefined:30s} {file_name}")

                else:
                    print(f"\nConverting file for {vunet_id:30s} {file_name}")


                if os.path.isfile(file_path.replace(".ipynb", ".html").strip('"')):
                    
     
                    print("File already converted; skipping")
                else:
                    os.system(f'jupyter nbconvert --to html {file_path}')
                    print(f"File for {vunet_id} done")
            

            if send_email:
                    
                if vunet_id is None:
                    print(f"Unable to get VUnet ID for {vunet_id}; email not sent\n")
                    list_of_unsent_emails.append(file_path)
                    continue

                print("Sending email to", vunet_id)

                if len(vunet_id) != 6:
                    print(f"VUnet ID for {vunet_id} is invalid; email not sent\n")
                    list_of_unsent_emails.append(file_path)
                    continue
                
                student = Student(vunet_id=vunet_id)
                email_address = student.email_address

                if email_address is None:
                    print(f"Unable to get email address for {vunet_id}; email not sent\n")
                    list_of_unsent_emails.append(file_path)
                    continue

                if go_to_latest == False:
                    if vunet_id == go_to_vunet_id:
                        go_to_latest = True
                    else:
                        print("nvm lmao")
                        continue
                
                print("Email address:", email_address, "\n")

                if test_mode:
                    email_address = ta_email

                mail = EmailMessage()
                mail["From"] = ta_email
                mail["To"] = email_address
                mail["Cc"] = ta_email

                mail["Subject"] = f'{assignment_number} Feedback and Results'

                message= f'''Dear student,

Please find attached your graded {assignment_number} with appropriate feedback. Details about the inspection hour will follow soon.

Kind regards and have a great day,
{ta_name}'''
                mail.set_content(message)

                attachment = os.path.join(dir_name, file_name.replace(".ipynb", ".html"))
                with open(attachment, "rb") as f:
                    mail.add_attachment(
                    f.read(),
                    filename=f"graded_test_{student.vunet_id}.html",
                    maintype="html",
                    subtype="html"
                )
                
                smtp = smtplib.SMTP("mails.vu.nl", port=587)
                smtp.starttls()
                smtp.login(ta_email, ta_email_password)
                smtp.sendmail(ta_email, [email_address, ta_email], mail.as_string())

                try: 
                    smtp.quit()
                
                except:
                    print("smtp.quit() failed. Idk why it does this. \n")

                list_of_sent_emails.append(vunet_id)

                if test_mode:
                    return
                    

    if file_convert:
        print("\nDone converting files")
    if send_email:
        print("\nDone sending emails")
        print("List of unsent emails: ", list_of_unsent_emails)
        print(f"Number of sent emails: {len(list_of_sent_emails)}")

# YOU DO WORK HERE

Change TA_EMAIL to your email, and TA_PASSWORD to your password. Change TA_NAME to your name (for "sincerely, {yourname}"), and ASSIGNMENT_NUMBER to the correct assignment nimber.

In [ ]:
# youraddress@student.vu.nl
TA_EMAIL = "F.M.LASTNAME@student.vu.nl"
TA_PASSWORD = "PASSWORD"

TA_NAME = "NAME"

# Format as "Assignment 2" or "Midterm"
ASSIGNMENT_NUMBER = "Resit Exam"

# If true, ends the first email to yourself instead of the first student, then stops the program
# Change this from true to false when you launch this
TEST_MODE = False

# How vunet ids are gotten. "FOLDER" for folders, "SCRAPE" for scraping the files. "SCRAPE" is more universal, but slower.
ID_MODE = "FOLDER"

htmlConvert(TA_NAME, TA_EMAIL, TA_PASSWORD, ASSIGNMENT_NUMBER, file_convert = True, send_email=True, test_mode=TEST_MODE, id_mode = ID_MODE)